### 역지오코딩

- 포트홀 발생하지 않은 지역의 자치구, 행정동 채워넣기

In [2]:
import pandas as pd
import numpy as np
import pickle

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
final4 = pd.read_pickle('/content/drive/MyDrive/DATA_전처리/pothole_final4.pickle')
final4['index'] = final4.index
final4.head()

,자치구,행정동,경도,위도,차선수,승용차,버스,트럭,총교통량,중대형차량 교통량,포트홀 존재여부,행정동_정규화,평균_건물연령,index
0,종로구,종로1가,126.979625,37.570080,3.0,22007.0,1728.0,2352.0,26087.0,0.156400,1,종로1가,29.5,0
1,NaN,NaN,126.981618,37.570169,3.0,22007.0,1728.0,2352.0,26087.0,0.156400,0,NaN,NaN,1
2,NaN,NaN,126.982593,37.570176,3.0,22007.0,1728.0,2352.0,26087.0,0.156400,0,NaN,NaN,2
3,종로구,종로1가,126.979760,37.570214,4.0,28631.0,1872.0,2959.0,33462.0,0.144373,1,종로1가,29.5,3
4,종로구,종로1가,126.979760,37.570214,4.0,28631.0,1872.0,2959.0,33462.0,0.144373,1,종로1가,29.5,4


In [ ]:
### 역지오코딩해서 자치구, 행정동 넣기
!pip install pandas openpyxl
!pip install geokakao
import geokakao as gk # 지오코딩
import requests # 역지오코딩
import re

from google.colab import userdata
api = userdata.get('KAKAO_API_KEY')

In [ ]:
no_pothole = final4[final4['포트홀 존재여부'] == 0]
no_pothole.head()

,자치구,행정동,경도,위도,차선수,승용차,버스,트럭,총교통량,중대형차량 교통량,포트홀 존재여부,행정동_정규화,평균_건물연령,index
1,NaN,NaN,126.981618,37.570169,3.0,22007.0,1728.0,2352.0,26087.0,0.156400,0,NaN,NaN,1
2,NaN,NaN,126.982593,37.570176,3.0,22007.0,1728.0,2352.0,26087.0,0.156400,0,NaN,NaN,2
15,NaN,NaN,126.982593,37.570284,4.0,23172.0,2736.0,2149.0,28057.0,0.174110,0,NaN,NaN,15
16,NaN,NaN,126.983343,37.570278,3.0,26544.0,2466.0,2851.0,31861.0,0.166881,0,NaN,NaN,16
29,NaN,NaN,126.983342,37.570170,3.0,19442.0,1905.0,2291.0,23638.0,0.177511,0,NaN,NaN,29


In [ ]:
### 역지오코딩
def convert_coordinates_to_address(latitude, longitude):
    y, x = str(latitude), str(longitude)
    url = f'https://dapi.kakao.com/v2/local/geo/coord2address.json?x={x}&y={y}'
    headers = {'Authorization': f'KakaoAK {api}'}

    try:
        response = requests.get(url, headers = headers)
        response.raise_for_status()
        result = response.json()

        if result.get("documents"):
            address_info = result["documents"][0].get("address")
            if address_info:
                return address_info.get("address_name")
            else:
                return "주소 못 찾았어요"

        else:
            return "주소 못 찾았다구요"

    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None

In [ ]:
print(convert_coordinates_to_address(37.570169, 126.981618))

서울 종로구 종로1가 54


In [ ]:
### 동 : ~동, ~가 부분 찾아오기
def get_dong(address):
    find = re.search(r"\b(\w+[동가])\b", address)
    if find:
        dong = find.group(1)
        return(dong)
    else:
        return(np.nan)

### 구 : ~구 부분 찾아오기
def get_gu(address):
    find = re.search(r"\b(\w+구)\b", address)
    if find:
        gu = find.group(1)
        return(gu)
    else:
        return(np.nan)

In [ ]:
def add_dong_gu_to_dataframe(df, latitude_col, longitude_col):
    dongs = []
    gus = []
    loop = 0

    for lat, lon in zip(df[latitude_col], df[longitude_col]):
        address = convert_coordinates_to_address(lat, lon)

        if address:
            try:
                dong = get_dong(address)
            except:
                dong = np.nan
            try:
                gu = get_gu(address)
            except:
                gu = np.nan
        else:
            dong = np.nan
            gu = np.nan

        loop += 1

        dongs.append(dong)
        gus.append(gu)

        if loop % 1000 == 0:
            print(f"{loop}개 완료")

    df["행정동"] = dongs
    df["자치구"] = gus
    return df

In [ ]:
no_pothole_filled = no_pothole.copy()
add_dong_gu_to_dataframe(no_pothole_filled,"위도","경도")

1000개 완료
2000개 완료


In [ ]:
no_pothole_filled.head()

,자치구,행정동,경도,위도,차선수,승용차,버스,트럭,총교통량,중대형차량 교통량,포트홀 존재여부,행정동_정규화,평균_건물연령,index
1,종로구,종로1가,126.981618,37.570169,3.0,22007.0,1728.0,2352.0,26087.0,0.156400,0,NaN,NaN,1
2,종로구,종로1가,126.982593,37.570176,3.0,22007.0,1728.0,2352.0,26087.0,0.156400,0,NaN,NaN,2
15,종로구,종로1가,126.982593,37.570284,4.0,23172.0,2736.0,2149.0,28057.0,0.174110,0,NaN,NaN,15
16,종로구,종로1가,126.983343,37.570278,3.0,26544.0,2466.0,2851.0,31861.0,0.166881,0,NaN,NaN,16
29,종로구,종로1가,126.983342,37.570170,3.0,19442.0,1905.0,2291.0,23638.0,0.177511,0,NaN,NaN,29


In [ ]:
no_pothole_filled[no_pothole_filled['행정동'] == '주소가']
no_pothole_filled['행정동'] = no_pothole_filled['행정동'].replace({"주소가": np.nan})

In [ ]:
no_pothole_filled.isna().sum()

,0
자치구,124
행정동,56
경도,0
위도,0
차선수,0
승용차,0
버스,0
트럭,0
총교통량,0
중대형차량 교통량,0


자치구에 124개 결측치, 행정동에 56개 결측치 존재

In [ ]:
no_pothole_filled[no_pothole_filled['자치구'].isna()]

,자치구,행정동,경도,위도,차선수,승용차,버스,트럭,총교통량,중대형차량 교통량,포트홀 존재여부,행정동_정규화,평균_건물연령,index
969,NaN,NaN,127.023289,37.575879,6.0,21323.0,1039.0,1906.0,24268.0,0.121353,0,NaN,NaN,969
1019,NaN,NaN,127.008272,37.557880,8.0,28061.0,755.0,2575.0,31391.0,0.106081,0,NaN,NaN,1019
1374,NaN,NaN,127.009783,37.569793,8.0,22965.0,1281.0,1965.0,26211.0,0.123841,0,NaN,NaN,1374
6554,NaN,NaN,127.030638,37.609184,6.0,22330.0,874.0,2396.0,25600.0,0.127734,0,NaN,NaN,6554
8763,NaN,장암동,127.051649,37.687030,2.0,2285.0,366.0,540.0,3191.0,0.283924,0,NaN,NaN,8763
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30734,NaN,NaN,126.799236,37.590159,2.0,12840.0,354.0,346.0,13540.0,0.051699,0,NaN,NaN,30734
30735,NaN,NaN,126.968288,37.697341,4.0,10769.0,234.0,1804.0,12807.0,0.159132,0,NaN,NaN,30735
30736,NaN,NaN,126.971068,37.706611,4.0,10769.0,234.0,1804.0,12807.0,0.159132,0,NaN,NaN,30736
30737,NaN,NaN,126.970602,37.703435,4.0,10769.0,234.0,1804.0,12807.0,0.159132,0,NaN,NaN,30737


In [ ]:
convert_coordinates_to_address(37.575879, 127.023289)

'주소 못 찾았다구요'

In [ ]:
missing = no_pothole_filled[no_pothole_filled['자치구'].isna() | no_pothole_filled['행정동'].isna()]
missing['위도경도'] = missing['위도'].astype(str) + ', ' + missing['경도'].astype(str)
missing.head()
missing.to_csv("/content/drive/MyDrive/Pothole 결측치/missing_new.csv", encoding='cp949')

<ipython-input-55-9ec5da9746a4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing['위도경도'] = missing['위도'].astype(str) + ', ' + missing['경도'].astype(str)


#### 합치기

In [1]:
pothole_id = pd.read_pickle('/content/drive/MyDrive/DATA_전처리/pothole_id.pickle')

NameError: name 'pd' is not defined